# Flood Map Prediction Using a Trained U-Net Model and Input Raster SAR Data
This notebook demonstrates the process of predicting flood maps using a pre-trained U-Net model and input raster SAR (Synthetic Aperture Radar) data. Specifically, Sentinel-1 SAR data is utilized to predict surface water and flooded areas.

The steps are as follows:

1. Load the Pre-Trained U-Net Model: The first step is to load the trained U-Net model, which will be used for making predictions.

2. Import and Preprocess Sentinel-1 SAR Data: The input SAR data (in .tif format) is loaded, and the image is divided into smaller 256x256 pixel chunks, each containing 4 bands. These chunks are generated because the U-Net model was trained on 256x256 pixel images with 4 bands (corresponding to the Sentinel-1 radar channels). This preprocessing step ensures that the input data is compatible with the model's requirements.

3. Predict Flood Areas: The preprocessed Sentinel-1 image chunks are passed through the trained U-Net model to predict water/flooded pixels. The model generates a classified output for each chunk, indicating flooded and non-flooded areas. These predicted chunks are then saved in a separate folder.

4. Reconstruct the Flood Map: The classified chunks are mosaicked together to reconstruct the final flood map, which represents the predicted surface water and flooded regions over the area covered by the input SAR data.

# Load Unet trained model

In [5]:
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import segmentation_models as sm

# Load the saved model
model = load_model(r'D:\BD_flood_upscale_data\scripts and trained model\trained_model.h5', custom_objects={'iou_score': sm.metrics.IOUScore(threshold=0.5)})

# Load the input SAR data (in .tif format) , and divide the image into smaller 256x256 pixel chunks

In [ ]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.plot import reshape_as_image
from sklearn.preprocessing import MinMaxScaler

# Parameters
root_directory = r'D:\BD_flood_upscale_data\data\Bogura'  # Root directory containing the images
output_directory = r'D:\BD_flood_upscale_data\chunks_image\Bogura'  # Directory to save the chunks
patch_size = 256  # Define the patch size for each chunk
scaler = MinMaxScaler()

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Initialize an empty dataset list
image_dataset = []

# Walk through the root directory to find images
for path, subdirs, files in os.walk(root_directory):
    dirname = path.split(os.path.sep)[-1]
    
    # Check if the folder is 'Images'
    if dirname == 'Images':
        images = [f for f in files if f.endswith(".tif")]
        
        for image_name in images:
            with rasterio.open(os.path.join(path, image_name)) as src:
                width = src.width
                height = src.height
                profile = src.profile  # Get the profile for saving
                
                # Loop over the image dimensions to create chunks
                for row in range(0, height, patch_size):
                    for col in range(0, width, patch_size):
                        # Define a window for the chunk
                        window = Window(col, row, min(patch_size, width - col), min(patch_size, height - row))
                        image_data = src.read(window=window)
                        image_data = reshape_as_image(image_data)
                        
                        # Process only if the chunk is the correct size
                        if image_data.shape[0] == patch_size and image_data.shape[1] == patch_size:
                            # Scale the image data
                            image_data_scaled = scaler.fit_transform(image_data.reshape(-1, image_data.shape[-1])).reshape(image_data.shape)
                            image_dataset.append(image_data_scaled)
                            
                            # Update profile for saving each chunk
                            profile.update({
                                "height": patch_size,
                                "width": patch_size,
                                "transform": rasterio.windows.transform(window, src.transform),
                                "dtype": 'float32',  # Assuming we want to save in 8-bit format
                                "count": image_data.shape[2]  # Number of channels
                            })
                            
                            # Save each chunked image
                            output_path = os.path.join(output_directory, f"{image_name}_chunk_{row}_{col}.tif")
                            with rasterio.open(output_path, 'w', **profile) as dst:
                                dst.write((image_data_scaled * 255).astype(np.uint8).transpose(2, 0, 1))  # Convert to 8-bit format for saving
                            
                            print(f"Saved chunk at: {output_path}")

# Convert image_dataset to numpy array if needed
image_dataset = np.array(image_dataset)


c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
C:\Users\vtiwari\AppData\Local\Temp\ipykernel_14976\2208751437.py:60: RuntimeWarning: invalid value encountered in cast
  dst.write((image_data_scaled * 255).astype(np.uint8).transpose(2, 0, 1))  # Convert to 8-bit format for saving
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
C:\Users\vtiwari\AppData\Local\Temp\ipykernel_14976\2208751437.py:6

Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_0.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_256.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_512.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_768.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_1024.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_1280.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_1536.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_1792.tif


C:\Users\vtiwari\AppData\Local\Temp\ipykernel_14976\2208751437.py:60: RuntimeWarning: invalid value encountered in cast
  dst.write((image_data_scaled * 255).astype(np.uint8).transpose(2, 0, 1))  # Convert to 8-bit format for saving
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
C:\Users\vtiwari\AppData\Local\Temp\ipykernel_14976\2208751437.py:60: RuntimeWarning: invalid value encountered in cast
  dst.write((image_data_scaled * 255).astype(np.uint8).transpose(2, 0, 1))  # Convert to 8-bit format for saving
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\

Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_2048.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_2304.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_2560.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_2816.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_3072.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_3328.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_3584.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_3840.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_4096.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_0_4352.tif


C:\Users\vtiwari\AppData\Local\Temp\ipykernel_14976\2208751437.py:60: RuntimeWarning: invalid value encountered in cast
  dst.write((image_data_scaled * 255).astype(np.uint8).transpose(2, 0, 1))  # Convert to 8-bit format for saving
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
C:\Users\vtiwari\AppData\Local\Temp\ipykernel_14976\2208751437.py:60: RuntimeWarning: invalid value encountered in cast
  dst.write((image_data_scaled * 255).astype(np.uint8).transpose(2, 0, 1))  # Convert to 8-bit format for saving
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\

...

Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_6912.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_7168.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_7424.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_7680.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_7936.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_8192.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_8448.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_8704.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_8960.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_9216.tif
Saved chunk at: D:\BD_flood_upscale_data

c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
C:\Users\vtiwari\AppData\Local\Temp\ipykernel_14976\2208751437.py:60: RuntimeWarning: invalid value encountered in cast
  dst.write((image_data_scaled * 255).astype(np.uint8).transpose(2, 0, 1))  # Convert to 8-bit format for saving
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
C:\Users\vtiwari\AppData\Local\Temp\ipykernel_14976\2208751437.py:6

Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_9728.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_9984.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_10240.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_10496.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_10752.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_11008.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_11264.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_11520.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_11776.tif
Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_12032.tif


C:\Users\vtiwari\AppData\Local\Temp\ipykernel_14976\2208751437.py:60: RuntimeWarning: invalid value encountered in cast
  dst.write((image_data_scaled * 255).astype(np.uint8).transpose(2, 0, 1))  # Convert to 8-bit format for saving
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
C:\Users\vtiwari\AppData\Local\Temp\ipykernel_14976\2208751437.py:60: RuntimeWarning: invalid value encountered in cast
  dst.write((image_data_scaled * 255).astype(np.uint8).transpose(2, 0, 1))  # Convert to 8-bit format for saving
c:\Users\vtiwari\.conda\envs\unet\lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\

Saved chunk at: D:\BD_flood_upscale_data\chunks_image\Barisal\Barishal.tif_chunk_14336_12288.tif


# APPLY LOADED UNET MODEL on Sentinel-1 SAR chuncks to predict water and non water class

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import rasterio
from rasterio.plot import reshape_as_image
import segmentation_models as sm

# Parameters
input_folder = r'D:\BD_flood_upscale_data\chunks_image\Bogura'  # Folder with input image chunks
output_folder = r'D:\BD_flood_upscale_data\chunks_predicted\Bogura'  # Folder to save individual predicted images
IMG_HEIGHT = 256  # Model input height (and chunk height)
IMG_WIDTH = 256   # Model input width (and chunk width)

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the saved model with custom IOU score
#model = load_model(r'D:\BD_flood_upscale_data\trained_model.h5', custom_objects={'iou_score': sm.metrics.IOUScore(threshold=0.5)})

# Retrieve chunk files in input folder
chunk_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

# Process each chunk, apply prediction, and save the predicted mask as an individual image
for image_file in chunk_files:
    try:
        # Load the image chunk
        with rasterio.open(os.path.join(input_folder, image_file)) as src:
            image_data = src.read()  # Read image data
            image_data = reshape_as_image(image_data)  # Reshape to (height, width, channels)
            
            # Resize the image to model input size
            image_resized = tf.image.resize(image_data, (IMG_HEIGHT, IMG_WIDTH))
            image_resized = np.expand_dims(image_resized, axis=0)  # Add batch dimension
            
            # Predict the mask
            predicted_mask = model.predict(image_resized)
            predicted_mask_classes = np.argmax(predicted_mask, axis=-1).squeeze()  # Get the class with the highest score

            # Ensure the predicted mask is of type 'uint8' for saving
            predicted_mask_classes = predicted_mask_classes.astype(np.uint8)

            # Debugging: Print shape of predicted mask to verify
            print(f"Predicted mask shape for {image_file}: {predicted_mask_classes.shape}")

        # Construct the output file path for the predicted mask
        output_file_path = os.path.join(output_folder, f'predicted_{image_file}')

        # Use metadata from the original chunk for geospatial info
        with rasterio.open(os.path.join(input_folder, image_file)) as src:
            profile = src.profile
            profile.update(
                driver='GTiff',
                count=1,  # Single band for mask
                dtype='uint8',  # Changed from 'byte' to 'uint8'
                compress='lzw'  # Compression for output file
            )

        # Save the predicted mask as a new TIFF file
        with rasterio.open(output_file_path, 'w', **profile) as dst:
            dst.write(predicted_mask_classes, 1)  # Write the mask to the first band
            print(f"Saved predicted mask for {image_file} to {output_file_path}")

    except Exception as e:
        print(f"Error processing {image_file}: {e}")

print(f"All predicted images saved in {output_folder}")


1/1 [==============================] - 1s 604ms/step
Predicted mask shape for Barishal.tif_chunk_0_0.tif: (256, 256)
Saved predicted mask for Barishal.tif_chunk_0_0.tif to D:\BD_flood_upscale_data\chunks_predicted\Barisal\predicted_Barishal.tif_chunk_0_0.tif
1/1 [==============================] - 0s 36ms/step
Predicted mask shape for Barishal.tif_chunk_0_1024.tif: (256, 256)
Saved predicted mask for Barishal.tif_chunk_0_1024.tif to D:\BD_flood_upscale_data\chunks_predicted\Barisal\predicted_Barishal.tif_chunk_0_1024.tif
1/1 [==============================] - 0s 37ms/step
Predicted mask shape for Barishal.tif_chunk_0_10240.tif: (256, 256)
Saved predicted mask for Barishal.tif_chunk_0_10240.tif to D:\BD_flood_upscale_data\chunks_predicted\Barisal\predicted_Barishal.tif_chunk_0_10240.tif
1/1 [==============================] - 0s 36ms/step
Predicted mask shape for Barishal.tif_chunk_0_10496.tif: (256, 256)
Saved predicted mask for Barishal.tif_chunk_0_10496.tif to D:\BD_flood_upscale_data\

# RECOMBINE predicited classified images to reconstruct classified image (MOSAIC)

In [ ]:
import os
import numpy as np
import rasterio
from rasterio.plot import reshape_as_image
from rasterio.windows import Window

# Parameters
input_directory = r'D:\BD_flood_upscale_data\chunks_predicted\Bogura'  # Directory containing the image chunks
output_image_path = r'D:\BD_flood_upscale_data\Mosaic\Bogura_classified_mosaic.tif'  # Path to save the recombined image


import os
import glob
import rasterio
from rasterio.merge import merge

def mosaic_rasters(input_folder, output_file):
    # Use glob to find all .tif files in the input folder
    tif_files = glob.glob(os.path.join(input_folder, "*.tif"))
    
    # Open all the raster files
    src_files_to_mosaic = []
    for tif_file in tif_files:
        src = rasterio.open(tif_file)
        src_files_to_mosaic.append(src)

    # Merge the rasters
    mosaic, out_transform = merge(src_files_to_mosaic)

    # Get metadata of the first raster (to keep the same for the output)
    out_meta = src_files_to_mosaic[0].meta.copy()

    # Update metadata with the new size, transform, and CRS
    out_meta.update({
        "driver": "GTiff",
        "count": mosaic.shape[0],  # Number of bands
        "width": mosaic.shape[2],  # Width of the mosaic
        "height": mosaic.shape[1], # Height of the mosaic
        "transform": out_transform
    })

    # Write the mosaic to the output file
    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(mosaic)

    # Close the source rasters
    for src in src_files_to_mosaic:
        src.close()

# Example usage
mosaic_rasters(input_directory, output_image_path)

